Crear categorías con funciones de fila

Has podido realizar un análisis simplificado sobre la correlación entre la puntuación de la crítica de un juego y su éxito comercial utilizando una función creada por ti mismo y el método apply(). Tu función personalizada tomó un solo argumento, así que nada más tuviste que llamar a apply() en la columna que contenía los valores que querías usar como input.

Pero, ¿qué pasa si queremos crear categorías con base en los valores de más de una columna? En ese caso, podemos escribir una función que toma una fila entera como input y extrae los valores que necesitamos para crear categorías nuevas.

Demostraremos esto con el conjunto de datos de los videojuegos. Esta vez vamos a simplificar las cosas y primero nos vamos a deshacer de todas las filas que no tengan valores:

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')

#remove all missing valued from the whole data frame
df.dropna(inplace=True)
df.info()

"""<class 'pandas.core.frame.DataFrame'>
Int64Index: 7943 entries, 0 to 16707
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   name             7943 non-null   object
 1   platform         7943 non-null   object
 2   year_of_release  7943 non-null   float64
 3   genre            7943 non-null   object
 4   publisher        7943 non-null   object
 5   developer        7943 non-null   object
 6   na_sales         7943 non-null   float64
 7   eu_sales         7943 non-null   float64
 8   jp_sales         7943 non-null   float64
 9   critic_score     7943 non-null   float64
 10  user_score       7943 non-null   object
dtypes: float64(5), object(6)
memory usage: 744.7+ KB"""

Hemos descartado muchas filas, pero ahora no tenemos que preocuparnos por que falten valores. Además, siempre podemos agregarlas de nuevo después del análisis preliminar para ver cómo eso afecta nuestros resultados.

Ahora, vamos a escribir una función llamada era_sales_group() que coloque los juegos en las siguientes categorías según el año de lanzamiento y las ventas totales:

- Los que se lanzaron antes del 2000 y que tienen menos de $1 millón en ventas irán a la categoría 'retro'.

- Los que se lanzaron entre el 2000 y el 2009 (inclusive) con menos de $1 millón de ventas irán a la categoría 'modern'.

- Los que se lanzaron antes del 2010 y que tienen $1 millón o más en ventas irán a la categoría 'classic'.

- Los que se lanzaron a partir del 2010 y que tienen menos de $1 millón en ventas irán a la categoría 'recent'.

- Los que se lanzaron a partir del 2010 y que tienen $1 millón o más en ventas irán a la categoría 'big hit'.

Así es como se verá la función junto con el output de muestra:



In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')

df.dropna(inplace=True)

def era_sales_group(row):
    """
    La función devuelve una categoría de juegos según el año de lanzamiento y las ventas totales mediante las siguientes reglas:
    —'retro'   para año < 2000 y las ventas totales < $1 million
    —'modern'  para 2000 <= año < 2010 y las ventas totales < $1 million
    —'recent'  para año >= 2010 y las ventas totales < $1 million
    —'classic' para año < 2010 y las ventas totales >= $1 million
    —'big hit' para año >= 2010 y las ventas totales >= $1 million
    """

    year = row['year_of_release']
    na_sales = row['na_sales']
    eu_sales = row['eu_sales']
    jp_sales = row['jp_sales']
    
    total_sales = na_sales + eu_sales + jp_sales
    
    if year < 2000:
        if total_sales < 1:
            return 'retro'
        else:
            return 'classic'
    if year < 2010:
        if total_sales < 1:
            return 'modern'
        else:
            return 'classic'
    if year >= 2010:
        if total_sales < 1:
            return 'recent'
        else:
            return 'big hit'

row = df.iloc[0] # usa la primera fila como input de ejemplo

print(row)
print()
print('Este juego es', era_sales_group(row))

"""
name               Wii Sports
platform                  Wii
year_of_release        2006.0
genre                  Sports
publisher            Nintendo
developer            Nintendo
na_sales                41.36
eu_sales                28.96
jp_sales                 3.77
critic_score             76.0
user_score                  8
Name: 0, dtype: object

This game is classic"""


Al igual que la función era_group() de la lección pasada, era_sales_group() tiene un parámetro: row. Sin embargo, en este caso se espera que row sea una sola fila entera del DataFrame.

Esta función requiere cuatro valores de la fila para hacer la categorización: uno para el año de lanzamiento y tres para calcular las ventas totales. Observa que la función usa nombres de columnas de df para extraer estos valores, así que no funcionará en otro DataFrame con columnas diferentes, sino que depende de los datos en vg_sales.csv.

Usamos df.iloc[0] para tomar la primera fila de df con el objetivo de probar nuestra función, lo que produjo el output 'classic'. Si revisamos las reglas de categorización, esto tiene sentido. Wii Sports fue lanzado antes del 2010 y ciertamente alcanzó más de $1 millón, ¡incluso mucho más!

Probar la función con filas personalizadas

Hemos probado era_sales_group() para el caso de 'classic', ¿pero qué pasa con las otras cuatro categorías? Sería una tarea tediosa buscar en los datos las filas correspondientes a cada caso. Afortunadamente, podemos crear nuestras propias filas con los valores que queramos al convertir una lista de nombres de columna y una lista de valores de fila en un objeto tipo Series:

In [ ]:
column_names = ['year_of_release', 'na_sales', 'eu_sales', 'jp_sales']
row_values = [2000, 0.1, 0.25, 0]

row = pd.Series(row_values, index=column_names)

print(row)
print()
print('Este juego es', era_sales_group(row))

"""
year_of_release    2000.00
na_sales              0.10
eu_sales              0.25
jp_sales              0.00
dtype: float64

This game is modern"""

Otra ventaja de probar la función de esta manera es que no necesitamos incluir valores para las columnas que no use nuestra función. En este caso, nuestra fila solo tiene valores para las columnas 'year_of_release', 'na_sales', 'eu_sales' y 'jp_sales'.

Para terminar las pruebas, vamos a usar la función con varios valores de input que nosotros creemos:

In [ ]:
cols = ['year_of_release', 'na_sales', 'eu_sales', 'jp_sales']

row_1 = pd.Series([1989, 0, 0, 0.6], index=cols) # expect 'retro'
row_2 = pd.Series([1989, 1, 2, 0], index=cols)   # expect 'classic'
row_3 = pd.Series([2006, 0.3, 0, 0], index=cols) # expect 'modern'
row_4 = pd.Series([2020, 0, 0.4, 0], index=cols) # expect 'recent'
row_5 = pd.Series([2020, 1, 1, 1], index=cols)   # expect 'big hit'

print(row_1, row_2, row_3, row_4, row_5, sep='\n\n')
print()

rows = [row_1, row_2, row_3, row_4, row_5]

for row in rows:
    print('Este juego es', era_sales_group(row))


"""
year_of_release    1989.0
na_sales              0.0
eu_sales              0.0
jp_sales              0.6
dtype: float64

year_of_release    1989
na_sales              1
eu_sales              2
jp_sales              0
dtype: int64

year_of_release    2006.0
na_sales              0.3
eu_sales              0.0
jp_sales              0.0
dtype: float64

year_of_release    2020.0
na_sales              0.0
eu_sales              0.4
jp_sales              0.0
dtype: float64

year_of_release    2020
na_sales              1
eu_sales              1
jp_sales              1
dtype: int64

This game is retro
This game is classic
This game is modern
This game is recent
This game is big hit"""

¡Parece que la función de categorización dio el resultado esperado! Para el último paso, vamos a usar la función para crear una columna nueva.

Crear una columna

Ahora que has comprendido mejor cómo trabajan las funciones, vamos a aprender a usarlas junto con apply() para crear nuevas columnas.

En este caso, queremos crear una columna llamada 'game_category' que categorice cada juego según el output de nuestra función era_sales_group(). Al igual que en la lección anterior, llamaremos al método apply(). No obstante, esta vez hay dos diferencias importantes:

El método apply() se llama en el DataFrame df entero en vez de en una sola columna.

Necesitamos usar el parámetro axis= al llamar al método apply().
De forma predeterminada, el parámetro axis= está en 0, lo que significa que apply() le pasa valores de columna a la función que se usa como input. En cambio, si queremos que apply() pase los valores de fila de la función, necesitamos configurar axis=1.

Entonces, se puede crear la nueva columna 'game_category' así:

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')
df.dropna(inplace=True)

def era_sales_group(row):
    """
    La función devuelve una categoría de juegos según el año de lanzamiento y las ventas totales mediante las siguientes reglas:
    —'retro'   para año < 2000 y las ventas totales < $1 million
    —'modern'  para 2000 <= año < 2010 y las ventas totales < $1 million
    —'recent'  para año >= 2010 y las ventas totales < $1 million
    —'classic' para año < 2010 y las ventas totales >= $1 million
    —'big hit' para año >= 2010 y las ventas totales >= $1 million
    """

    year = row['year_of_release']
    na_sales = row['na_sales']
    eu_sales = row['eu_sales']
    jp_sales = row['jp_sales']
    
    total_sales = na_sales + eu_sales + jp_sales
    
    if year < 2000:
        if total_sales < 1:
            return 'retro'
        else:
            return 'classic'
    if year < 2010:
        if total_sales < 1:
            return 'modern'
        else:
            return 'classic'
    if year >= 2010:
        if total_sales < 1:
            return 'recent'
        else:
            return 'big hit'

df['game_category'] = df.apply(era_sales_group, axis=1)
print(df.sample(5, random_state=321))

"""                                    name platform  year_of_release      genre  \
7654          Dave Mirra Freestyle BMX 2       XB           2001.0     Sports   
6815          Burnout 2: Point of Impact       XB           2003.0     Racing   
9824   NPPL: Championship Paintball 2009     X360           2008.0    Shooter   
34            Call of Duty: Black Ops II      PS3           2012.0    Shooter   
14940  Agatha Christie's The ABC Murders      PS4           2016.0  Adventure   

                   publisher        developer  na_sales  eu_sales  jp_sales  \
7654   Acclaim Entertainment     Z-Axis, Ltd.      0.15      0.04      0.00   
6815   Acclaim Entertainment  Criterion Games      0.18      0.05      0.00   
9824        Activision Value         FUN Labs      0.10      0.01      0.00   
34                Activision         Treyarch      4.99      5.73      0.65   
14940               Microids         Microids      0.01      0.01      0.00   

       critic_score  user_score game_category  
7654           76.0         8.2        modern  
6815           88.0         5.1        modern  
9824           44.0         7.3        modern  
34             83.0         5.3       big hit  
14940          67.0         6.2        recent
"""

Ahora vemos que logramos crear la nueva columna y podemos comenzar el análisis. Por ejemplo, podemos recurrir al método value_counts() para obtener el número de juegos que pertenecen a cada categoría:

In [ ]:
print(df['game_category'].value_counts())

"""
modern     3907
recent     1784
classic     748
big hit     407
retro        43
Name: game_category, dtype: int64"""

1.

Escribe una función que se llame avg_score_group() (grupo puntuación promedio) que tenga un parámetro llamado row. El parámetro row debe ser un objeto tipo Series de pandas. La función debe calcular la calificación promedio de cada juego, luego devolver una cadena que coloque cada uno en una de estas categorías:

- valor 'low' para promedios menores de 60.
- valor 'medium' para promedios de 60 a 79.
- valor 'high' (alto) para puntuaciones mayores a 80.

Para calcular la puntuación promedio, avg_score_group() debe tomar los valores de row con los nombres de columna 'critic_score' y 'user_score'. La fórmula para calcularlo es avg_score = (critic_score + user_score * 10) / 2.

Te dejamos las pruebas hechas, se debe imprimir low, medium y high, en ese orden.

In [ ]:
import pandas as pd
df = pd.read_csv('/datasets/vg_sales.csv')
df.dropna(inplace=True)

def avg_score_group(row):
    """
    La funcion devuelve puntuacion promedio segun la calificacion de los criticos y de los usuarios
    -valor 'low' para promedios menores de 60.
    -valor 'medium' para promedios de 60 a 79.
    -valor 'high' (alto) para puntuaciones mayores a 80"""
    
    critic_score = row['critic_score']
    user_score = row['user_score']
    
    avg_score = (critic_score + user_score * 10) / 2
    
    if avg_score < 60:
        return 'low'
    if avg_score > 60:
        if avg_score < 80:
            return 'medium'
        else:
            return 'high'# escribe tu función aquí

# parte de prueba a continuación, por favor no la cambies

col_names = ['critic_score', 'user_score']
test_low  = pd.Series([10, 1.0], index=col_names)
test_med  = pd.Series([65, 6.5], index=col_names)
test_high = pd.Series([99, 9.9], index=col_names)

rows = [test_low, test_med, test_high]

for row in rows:
    print(avg_score_group(row))


"""
low
medium
high"""

2.

Ahora es momento de poner a prueba tu nueva función. Crea tres filas personalizadas con estos nombres de variables y valores:

row_1: puntuación de la crítica de 66 y puntuación de los usuarios de 3.6.
row_2: puntuación de la crítica de 72 y puntuación de los usuarios de 8.1.
row_3: puntuación de la crítica de 99 y puntuación de los usuarios de 9.4.
Cada una de las variables de fila debe ser un objeto tipo Series con valores de índice 'critic_score' y 'user_score' para que avg_score_group() pueda extraer los valores correctos.

El precódigo define la función avg_score_group() del ejercicio anterior, aunque puede que se vea distinta a tu solución. Tu tarea es imprimir el resultado de llamar a avg_score_group() con cada uno de los inputs especificados.

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')
df.dropna(inplace=True)

def avg_score_group(row):
    critic_score = row['critic_score']
    user_score = row['user_score']
    
    avg_score = (critic_score + user_score * 10) / 2
    
    if avg_score < 60:
        return 'low'
    if avg_score < 80:
        return 'medium'
    if avg_score >= 80:
        return 'high'


# crea las filas de input de prueba aquí
column_names = ['critic_score', 'user_score']
row_1 = pd.Series([66, 3.6], index=column_names)
row_2 = pd.Series([72, 8.1], index=column_names)
row_3 = pd.Series([99, 9.4], index=column_names)
    
# imprime los resultados de llamar a la función con los input de prueba en orden
print(avg_score_group(row_1))
print(avg_score_group(row_2))
print(avg_score_group(row_3))

#output:
#low
#medium
#high